In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

In [ ]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [ ]:
# settings
MODEL = "yolov8x.pt"

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [2, 3, 5, 7]

In [ ]:
# create frame generator
generator = get_video_frames_generator('/content/drive/MyDrive/videos for system/2.mp4')
# create instance of BoxAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame)
detections = Detections(
    xyxy=results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)
# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]
# annotate and display frame
frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)

%matplotlib inline
show_frame_in_notebook(frame, (16, 16))

In [ ]:
#####هذا الكود
import cv2
from ultralytics import YOLO, solutions

model = YOLO("yolov8x.pt")
names = model.model.names

cap = cv2.VideoCapture("/content/drive/MyDrive/videos for system/2.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define region points
region_points = [(1000, 1000), (3000, 1000), (3000, 900), (1000, 900)]

# Video writer
video_writer = cv2.VideoWriter("/content/drive/MyDrive/videos for system/newelian2.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=region_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=5,
)

# Init speed-estimation obj
line_pts = [(1000, 950), (3000, 950)]
speed_obj = solutions.SpeedEstimator(
    reg_pts=line_pts,
    names=names,
    view_img=True,
    line_thickness=5,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False,classes=[2, 3, 5, 7])

    im0 = counter.start_counting(im0, tracks)
    im0 = speed_obj.estimate_speed(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import os
import tkinter as tk
from tkinter import ttk
from tkinter import font as tkfont

# Load the pre-trained ResNet50 model
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Define the car information
car_info = {
    "Hyundai i30 FastBack 2018": {
        "input_image_path": "s.png",
        "dataset_folder": "/content/drive/MyDrive/data_crop"
    },
    "Hyundai i30 Wagon 2013": {
        "input_image_path": "ima21ges.png",
        "dataset_folder": "/content/drive/MyDrive/data_crop"
    },
    "Škoda Octavia III 2017": {
        "input_image_path": "Captursswqe.PNG",
        "dataset_folder": "/content/drive/MyDrive/data_crop"
    },
     "kia rio": {
        "input_image_path": "/content/drive/MyDrive/ff.png",
        "dataset_folder": "/content/drive/MyDrive/data_crop"
    }
    # Add more car information as needed
}

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = img[..., ::-1]  # Convert BGR to RGB
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

def extract_features(img_path, model):
    img = preprocess_image(img_path)
    features = model.predict(img)
    return features.flatten()

def find_similar_images(input_image_path, dataset_folder, model, threshold=0.7):
    """
    Finds images in the dataset with a similarity score above the given threshold.

    Args:
        input_image_path (str): Path to the input image.
        dataset_folder (str): Path to the folder containing the dataset images.
        model (tf.keras.Model): The pre-trained model for feature extraction.
        threshold (float): The minimum similarity score for an image to be considered a match.

    Returns:
        list: A list of paths to images with similarity scores above the threshold.
    """
    input_features = extract_features(input_image_path, model)
    similar_images = []

    for img_file in os.listdir(dataset_folder):
        img_path = os.path.join(dataset_folder, img_file)
        if img_path.endswith(('.jpg', '.jpeg', '.png')):
            dataset_features = extract_features(img_path, model)
            similarity = cosine_similarity([input_features], [dataset_features])[0][0]
            if similarity >= threshold:
                similar_images.append((img_path, similarity))

    return similar_images

def process_image():
    selected_car = car_var.get()
    if selected_car in car_info:
        input_image_path = car_info[selected_car]["input_image_path"]
        dataset_folder = car_info[selected_car]["dataset_folder"]

        similar_images = find_similar_images(input_image_path, dataset_folder, model)

        if similar_images:
            plt.figure(figsize=(10, 5))
            for i, (img_path, similarity) in enumerate(similar_images):
                similar_img = cv2.imread(img_path)
                similar_img = cv2.cvtColor(similar_img, cv2.COLOR_BGR2RGB)
                plt.subplot(len(similar_images), 1, i+1)
                plt.title(f'Similar Image {i+1}\nSimilarity: {similarity:.4f}')
                plt.imshow(similar_img)
                plt.axis('off')
            error_label.config(text="Success.")
            plt.show()
            error_label.config(text="")  # Clear any previous error message
        else:
            error_label.config(text="No images found with similarity above 70%.")
    else:
        error_label.config(text="Please select a valid car.")

# Create the main window
root = tk.Tk()
root.title("Image Similarity Search")

# Set custom font for the application
custom_font = tkfont.Font(family="Arial", size=12, weight="bold")

# Create a dropdown list for car selection
car_var = tk.StringVar(root)
car_var.set(list(car_info.keys())[0])  # Set the default selection to the first car
car_dropdown = ttk.Combobox(root, textvariable=car_var, values=list(car_info.keys()), font=custom_font)
car_dropdown.grid(row=0, column=0, padx=10, pady=10)

# Create a button to trigger the image processing
process_button = tk.Button(root, text="Process Image", command=process_image, font=custom_font)
process_button.grid(row=1, column=0, padx=10, pady=10)

# Create a label to display any error messages
error_label = tk.Label(root, text="", fg="red", font=custom_font)
error_label.grid(row=2, column=0, padx=10, pady=10)

root.mainloop()

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter, speed_estimation
import cv2

model = YOLO("yolov8x.pt")
names = model.model.names

cap = cv2.VideoCapture("/content/drive/MyDrive/Video/Trim2.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
combined_video_writer = cv2.VideoWriter("combined_output2.avi",
                                        cv2.VideoWriter_fourcc(*'mp4v'),
                                        fps,
                                        (w, h))

region_points = [(500, 1100), (3500, 1100), (1080, 360), (20, 360)]
# Define line points for speed estimation
line_pts = [(500, 1100), (3500, 1100)]

# Init Object Counter
counter = object_counter.ObjectCounter()
counter.set_args(view_img=True,
                 reg_pts=region_points,
                 classes_names=model.names,
                 draw_tracks=True,
                 line_thickness=1,view_in_counts=True,view_out_counts=True)

# Init Speed Estimator
speed_obj = speed_estimation.SpeedEstimator()
speed_obj.set_args(reg_pts=line_pts,
                   names=names,
                   view_img=True)

while cap.isOpened():

    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    # Object counting
    im0_object_counting = counter.start_counting(im0, tracks)

    # Speed estimation
    im0_speed_estimation = speed_obj.estimate_speed(im0_object_counting, tracks)

    # Combine images
    combined_image = cv2.addWeighted(im0_object_counting, 0.5, im0_speed_estimation, 0.5, 0)

    combined_video_writer.write(combined_image)

cap.release()
combined_video_writer.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import cv2
import os

model = YOLO("yolov8x")
names = model.names

cap = cv2.VideoCapture("/content/drive/MyDrive/Trim.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

crop_dir_name = "/content/drive/MyDrive/data_crop3"
if not os.path.exists(crop_dir_name):
    os.mkdir(crop_dir_name)

# Video writer
video_writer = cv2.VideoWriter("object_cropping_output.avi",
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               fps, (w, h))

idx = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.predict(im0, show=False)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    clss = results[0].boxes.cls.cpu().tolist()
    annotator = Annotator(im0, line_width=2, example=names)

    if boxes is not None:
        for box, cls in zip(boxes, clss):
            idx += 1
            annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

            crop_obj = im0[int(box[1]):int(box[3]), int(box[0]):int(box[2])]

            cv2.imwrite(os.path.join(crop_dir_name, str(idx)+".png"), crop_obj)


    video_writer.write(im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows()


In [ ]:
# save pipe.pkl to output data folder
!cp -r /content/uy.avi /content/drive/MyDrive

In [ ]:
!zip -r dataset.zip ultralytics_crop/